In [9]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [10]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [11]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 6658}


6658

In [12]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 11781}


11781

In [13]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,afb8e20ce5734aa8b261aac412ca663a,acherisca,2022-03-15T14:37:24.199000Z,75534df4-f1dd-4717-9ef0-55ee6b73e58c,0,Jean Baptiste,Mona
1,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-15T14:37:03.863000Z,4f9d864c90774e0fb01961b4d087ec53,1,Dorsainville,Genise
2,8fd7f28e6c31484993c3153565feb429,cmariedanielle,2022-03-15T15:36:22.126000Z,0ffdf0a8b8b34f4cb265db6db7c0fab1,4,Compère,Veniat
3,ece16e9f4d2c468e961d2b56ff621a2e,myvelise,2022-03-15T14:36:00.055000Z,024c3854-7dd6-4fcc-a0ce-ff136e7baa45,0,Exantus,Montas Junior
4,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-15T14:35:11.612000Z,b375dd53dfe94d779567802fcc6f4ac3,3,Jean,Mana
...,...,...,...,...,...,...,...
6469,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
6470,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
6471,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
6472,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


6474

In [14]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
11776,8fd7f28e6c31484993c3153565feb429,516cbee8-2891-44e3-82e6-af6453338e0a,6,1,non,non,0ffdf0a8b8b34f4cb265db6db7c0fab1,child
11777,8fd7f28e6c31484993c3153565feb429,58d23134-b039-4c21-8c14-586e12859f12,5,1,non,non,0ffdf0a8b8b34f4cb265db6db7c0fab1,child
11778,8fd7f28e6c31484993c3153565feb429,db14e9a3-2f3d-4be4-bc55-240026dd4f74,3,2,non,non,0ffdf0a8b8b34f4cb265db6db7c0fab1,child
11779,8fd7f28e6c31484993c3153565feb429,1661ec34-01d0-4950-b5d7-9bb97287ff06,12,2,non,non,0ffdf0a8b8b34f4cb265db6db7c0fab1,child


11781

In [15]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [16]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.qty.sum()
result_hivmuso.reset_index(inplace=True)
result_hivmuso


,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",182
1,"(Gustave, firma.gustave@carisfoundationintl.org)",320
2,"(June007, jeanrony.valbrun@carisfoundationintl...",578
3,"(despeignes, roselord.despeignes@carisfoundati...",75
4,"(jfodney17, fodney.joseph@carisfoundationintl....",454
5,"(josy22, josy.serafils@carisfoundationintl.org)",319
6,"(judesonvOLMAR, judeson.volmar@carisfoundation...",135
7,"(marcakernstdubois, kernst.dubois@carisfoundat...",405
8,"(suzanne, suzanne.clerge@carisfoundationintl.org)",121
9,"(viola, viola.tombeau@carisfoundationintl.org)",14
